<a href="https://colab.research.google.com/github/Seongjin1225/AI_SCHOOL_9/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A0%84%EC%B2%98%EB%A6%AC%20%EB%AF%B8%EB%8B%88%20%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8/%EC%84%9C%EC%9A%B8_%EC%A7%80%ED%95%98%EC%B2%A0_%EC%A7%80%EB%8F%84_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문제 1. 불금 지하철 하차 히트맵 & 토요일 아침 복귀 인원 히트맵
# 문제 2. 위와 동일 but 상위 3개 하위 3개 동그라미 처리(red,blue)

In [ ]:
# 라이브러리 호출
import pandas as pd
import folium

# 데이터 불러오기 및 데이터 확인
data = pd.read_csv('/content/drive/MyDrive/AI스쿨 파일/seoul-metro-2021.logs.csv')
info = pd.read_csv('/content/drive/MyDrive/AI스쿨 파일/seoul-metro-station-info.csv')
data.head()
info.head()

# 필요한 컬럼만 추출하기
info = info[['station.code','geo.latitude','geo.longitude']]
info.head()

# 역 정보 기준으로 그룹화
data_sum = data.groupby('station_code').sum(numeric_only=False)  # 숫자형 데이터 타입 아닌거 있는 경우는 numeric_only=False 해줘야함
data_sum

# 데이터 합치기 위해 info의 인덱스 재설정
info = info.set_index('station.code')

# join 사용해 두 데이터 합치기
joined_data = data_sum.join(info)
joined_data.head()

,timestamp,people_in,people_out,geo.latitude,geo.longitude
station_code,,,,,
150,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,12697273,12109991,37.554648,126.972559
151,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,5997344,6030491,37.564718,126.977108
152,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,9638952,9284693,37.570161,126.982923
153,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,7704599,7090896,37.571607,126.991806
154,2021-01-01T05:00:00.000+09:002021-01-01T06:00:...,6651283,6609055,37.570926,127.001849


In [ ]:
# 위에서 문제에 필요한 사전 작업은 다 마무리 했음
# 문제 1 풀기

# 불금 데이터 추출하기(19시~)
friday_data = data[(pd.to_datetime(data.timestamp).dt.weekday == 4) & (pd.to_datetime(data.timestamp).dt.hour > 18)]
friday_data.head()

# 토요일 오전 5~8시 데이터 추출
saturday_data = data[(pd.to_datetime(data.timestamp).dt.weekday==5) & (pd.to_datetime(data.timestamp).dt.hour < 9)]
saturday_data.head()

# 각각의 데이터 그룹화
friday_group = friday_data.groupby('station_code').sum(numeric_only=False)
friday_group.head()

saturday_group = saturday_data.groupby('station_code').sum(numeric_only=False)
saturday_group.head()

# 각각의 데이터 합치기
friday_join = friday_group.join(info)
friday_join.head()

saturday_join = saturday_group.join(info)
saturday_join.head()

,timestamp,people_in,people_out,geo.latitude,geo.longitude
station_code,,,,,
150,2021-01-02T05:00:00.000+09:002021-01-02T06:00:...,99950,197738,37.554648,126.972559
151,2021-01-02T05:00:00.000+09:002021-01-02T06:00:...,22421,79892,37.564718,126.977108
152,2021-01-02T05:00:00.000+09:002021-01-02T06:00:...,26109,88952,37.570161,126.982923
153,2021-01-02T05:00:00.000+09:002021-01-02T06:00:...,30886,81073,37.571607,126.991806
154,2021-01-02T05:00:00.000+09:002021-01-02T06:00:...,23193,94704,37.570926,127.001849


In [ ]:
# 문제 1.
# 위에서 데이터 전부 완성했으니 히트맵으로 표시 ㄱㄱ
from folium.plugins import HeatMap
friday = folium.Map(location=[37.55, 126.98], zoom_start = 12)  # zoom_size - 좌표 위치에서 얼마나 확대된 화면이 첫 화면 될지(0~18)
HeatMap(data=friday_join[['geo.latitude','geo.longitude','people_in']]).add_to(friday)
friday

In [ ]:
# 토요일 데이터 히트맵으로 표현
saturday= folium.Map(location=[37.55, 126.98], zoom_start = 12)
HeatMap(data=saturday_join[['geo.latitude','geo.longitude','people_out']]).add_to(saturday)
saturday

## 문제 2번

In [ ]:
# 라이브러리 호출
import pandas as pd
import folium

# 데이터 불러오기 및 데이터 확인
data = pd.read_csv('/content/drive/MyDrive/AI스쿨 파일/seoul-metro-2021.logs.csv')
info = pd.read_csv('/content/drive/MyDrive/AI스쿨 파일/seoul-metro-station-info.csv')
data.head()
info.head()

# 필요한 컬럼만 추출하기
info = info[['station.code', 'station.name_full', 'geo.latitude','geo.longitude']]
info.head()

# 역 정보 기준으로 그룹화
data_sum = data.groupby('station_code').sum(numeric_only=False)  # 숫자형 데이터 타입 아닌거 있는 경우는 numeric_only=False 해줘야함
data_sum

# 데이터 합치기 위해 info의 인덱스 재설정
info = info.set_index('station.code')

# join 사용해 두 데이터 합치기
joined_data = data_sum.join(info)
joined_data.head()

# 2021년 1월 데이터만 추출
data_jan = data[(pd.to_datetime(data.timestamp).dt.year == 2021) & (pd.to_datetime(data.timestamp).dt.month == 1)]
jan_group = data_jan.groupby('station_code').sum(numeric_only=False)
jan_join = jan_group.join(info)

# 승차 상하위 3개 데이터
top_in = jan_join.sort_values(by='people_in')[-3:]
top_in
low_in = jan_join.sort_values(by='people_in')[:3]
low_in

# 하차 상하위 3개 데이터
top_out = jan_join.sort_values(by='people_out')[-3:]
top_out[-3:]
low_out = jan_join.sort_values(by='people_out')[:3]
low_out

,timestamp,people_in,people_out,station.name_full,geo.latitude,geo.longitude
station_code,,,,,,
321,2021-01-02T05:00:00.000+09:002021-01-02T06:00:...,0,0,충무로,37.561243,126.994280
2649,2021-01-07T05:00:00.000+09:002021-01-07T06:00:...,0,0,신내,37.612668,127.103213
2615,2021-01-02T05:00:00.000+09:002021-01-02T06:00:...,0,0,연신내,37.619001,126.921008


In [ ]:
# 승차 top 3 역 지도에 표시
top_in_map = folium.Map(location=[37.55, 126.98], zoom_start = 12)

for idx, row in top_in.iterrows():
    folium.Marker(location=[row['geo.latitude'], row['geo.longitude']], popup=row['station.name_full'],
                  icon=folium.Icon(color='blue',icon='star')).add_to(top_in_map)
top_in_map

In [ ]:
low_in_map = folium.Map(location=[37.55, 126.98], zoom_start = 12)

for idx, row in low_in.iterrows():
    folium.Marker(location=[row['geo.latitude'], row['geo.longitude']], popup=row['station.name_full'],
                  icon=folium.Icon(color='blue',icon='star')).add_to(low_in_map)
low_in_map

In [ ]:
# 하차 top 3 역 지도에 표시
top_out_map = folium.Map(location=[37.55, 126.98], zoom_start = 12)

for idx, row in top_out.iterrows():
    folium.Marker(location=[row['geo.latitude'], row['geo.longitude']], popup=row['station.name_full'],
                  icon=folium.Icon(color='blue',icon='star')).add_to(top_out_map)
top_out_map

In [ ]:
low_out_map = folium.Map(location=[37.55, 126.98], zoom_start = 12)

for idx, row in low_out.iterrows():
    folium.Marker(location=[row['geo.latitude'], row['geo.longitude']], popup=row['station.name_full'],
                  icon=folium.Icon(color='blue',icon='star')).add_to(low_out_map)
low_out_map